In [ ]:
# @title Class & Inheritence
class Person():
  amount = 0 # class variable, this will not be changed by Person objects
             # not unique for each created object/ same for every created object
  def __init__(self, name, age):
    self.name = name
    self.age = age
    Person.amount += 1

  def __del__(self):
    Person.amount -= 1

  def __str__(self):
    return f"This persons name is {self.name}, age is {self.age}, and {Person.amount}th person in line"

  def gets_older(self, years):
    self.age += years

class Worker(Person):

  def __init__(self, name, age, salary):
    super(Worker, self).__init__(name, age)
    self.salary = salary
    Person.amount += 1

  def __str__(self):
    text = super(Worker, self).__str__()
    text += f", salary {self.salary}"
    return text
     #return f"This persons name is {self.name}, age is {self.age}, and {Person.amount}th person in line and earns {self.salary}"

  def yearly_salary(self):
    return self.salary * 12

class Vector():
  def __init__(self, x, y):
    self.x = x
    self.y = y

  def __str__(self):
    return f"x:{self.x}, y:{self.y}"

  def __add__(self, other):
    return (self.x + other.x, self.y + other.y)

  def __sub__(self, other):
    return (self.x - other.x, self.y - other.y)

In [ ]:
# @title
person1 = Person('Mike', 30)
print(person1)
person2 = Person('bob', 20)
print(person2)

print(Person.amount)
#del person1
print(Person.amount)

This persons name is Mike, age is 30, and 1th person in line
This persons name is bob, age is 20, and 2th person in line
2
2


In [ ]:
# @title
worker1 = Worker('Henry', 40, 10000)
worker2 = Worker('Joe', 30, 5000)
print(worker1)
print(Person.amount)

This persons name is Henry, age is 40, and 4th person in line, salary 10000
4


In [ ]:
# @title
v1 = Vector(2,5)
v2 = Vector(3,3)
print(v1, v2)
v3 = v1 + v2
print(v3)

x:2, y:5 x:3, y:3
(5, 8)


In [ ]:
# @title Multithreading
# execute multiple tasks at the same time on multiple threads
# multiple thread in the same process share the same memory space; so they can communicate better
import threading

# useful cases example: video games where multiple processes like
# user input, sound, interaction, video render etc need to be
# executed simultaneously or parallaly

# the script that we are running is alread running on a 'main thread',
# and any additional threads that are defined and start() will run parallel to main thread
# so if we want a thread to stop and then run the next thread we will need join() method

In [1]:
# @title
def function1():
  for x in range(50):
    print(f"{x}: function1")

def function2():
  for x in range(50):
    print(f"{x}: function2")

t1 = threading.Thread(target = function1)
t2 = threading.Thread(target = function2)
t1.start()
#t1.join()
t2.start()
#t2.join()

# if we let all defined threads run in parallel, than they will
# so fast that the print statements may not show in order
print("text")

NameError: ignored

In [ ]:
# @title Synchronizing Threads
# we saw above that multithreading runs processes parallely.
# Now if we have a file that we are changing on one thread,
# and reading on another thread; then the values might get jumbled up since threads are running so fast parallely
# thus the threads will counteract each other
import time

x = 8192
lock = threading.Lock() # lock down access to thread where a function is executing


def double():
  global x, lock # if we want to manipulate a variable declared outside of the function

  lock.acquire() # this will try to acquire the thread, if its free.
  # if a thread is already locked by another function then we can not acquire it until it free/ done executing

  while x < 16384:
    x *= 2
    print(x)
    time.sleep(1)
  print("reached the max")
  lock.release()

def halve():
  global x, lock
  lock.acquire()
  while x > 1:
    x /= 2
    print(x)
    time.sleep(1)
  print("reached the minimum")
  lock.release()


t1 = threading.Thread(target=double)
t2 = threading.Thread(target=halve)

In [ ]:
# @title
# buggy here, try on PC
t2.start()
t1.start()

4096.0


In [ ]:
# @title
semaphore = threading.BoundedSemaphore(value=5)
# semaphore doesn't lock the resource completely, but limits the access
# so multiple thread can access a resource but not unlimited
def access(thread_number):
  print(f"{thread_number} is trying to access")
  semaphore.acquire()
  print(f"{thread_number} was granted access")
  time.sleep(10)
  print(f"{thread_number} is now releasing")
  semaphore.release()

for thread_number in range(1,11):
  t = threading.Thread(target=access, args=(thread_number,))
  t.start()
  time.sleep(1)

# in the output we see after 5 threads were granted access,
# further threads were not allowed access to the resource untill
# the previous thread access is released
# so the 6-10 were trying to access and was finally granted access
# when time=10sec passed, and thread1 released access

1 is trying to access
1 was granted access
2 is trying to access
2 was granted access
3 is trying to access
3 was granted access
4 is trying to access
4 was granted access
5 is trying to access
5 was granted access
6 is trying to access
7 is trying to access
8 is trying to access
9 is trying to access
10 is trying to access
1 is now releasing
6 was granted access


In [ ]:
# @title Events
event = threading.Event()

def myFunction():
  print("Waiting for event to trigger...\n")
  event.wait()
  print("WOW the event triggered!!\n")

t1 = threading.Thread(target=myFunction)

In [ ]:
# @title
t1.start()
x = input("So you want to trigger the event? (y/n)\n")
if x == "y":
  event.set()

# here we see that t1 thread was waiting,
# but main thread executed the 'input' command parallely to t1 thread

Waiting for event to trigger...

So you want to trigger the event? (y/n)
y
WOW the event triggered!!



In [ ]:
# @title Daemon Threads
# Daemon thread are threads that keep running in the background
# EVEN IF the main script is terminated,
# or EVEN IF the programme is waiting for a few threads to stop running, so the next threads can run
# so no one waits for Daemon threads, and Daemon threads wait for none
#

In [ ]:
# @title
path = "/content/text.txt"
text = ""

def readFile():
  global path, text
  while True: # endless loop
    with open(path, "r") as f:
      text = f.read()
    time.sleep(3)

def printloop():
  for x in range(30):
    print(text)
    time.sleep(1)

t1 = threading.Thread(target=readFile, daemon=True)
t2 = threading.Thread(target=printloop)


In [ ]:
t1.start()
t2.start()
# run it on pc, doesn't work in colab

Hello World!!


In [ ]:
# @title Queue
import queue

numbers = [10,20,30,40,50,60,70]
q = queue.Queue()
que = queue.LifoQueue()
qu = queue.PriorityQueue()
for num in numbers:
  # FIFO queue
  q.put(num)
  # LIFO queue/ stack
  que.put(num)

# Priorty Queue
qu.put((2, "Hello World"))
qu.put((11, 99))
qu.put((5, 7.5))
qu.put((1, True))

while not q.empty():
  print(q.get())
  # FIFO queue
print("-----------------")
while not que.empty():
    print(que.get())
  # LIFO queue/ stack
print("-----------------")
while not qu.empty():
  print(qu.get()[1])



In [ ]:
# @title Sockets and Network Programming
# socket is basically an end point to receive data
# Network programming - how client(socket) and server(socket) will interact with each other
# TCP, UDP, FTP, HTTP etc type of protocals
# TCP trasnfers accurate data, but slower (good for messsages)
# UDP transfers data faster, but risk losing some data (good for skype, multiplayer)
# also pick which IP I want to use from my pc
import socket


In [ ]:
# @title
#                (internet socket, tcp protocol)
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('127.0.0.1', 55555)) # run this on your local machine
s.listen() # socket is listening for any possible connections


'''This is the server script'''
while True():
  client, address = s.accept()
  # accept a client when client tries to connect to socket
  # store the client and its address
  print("Connected to ()".format(address))
  client.send("You are connected".encode()) # send feedback to client
  client.close() # close current client, so that we don't have unlimited clients running in background


In [ ]:
# @title
'''This is the client script'''
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('127.0.0.1', 55555)) # (Local_Host_IP_address, port)

message = s.receive(1024) # receive 1024 bytes
print(message.decode())


In [ ]:
# @title Database Programming
import sqlite3

# if there exists a database then connect to it, if it doesn't then create new database

connection = sqlite3.connect('/content/mydata.db')
cursor = connection.cursor() # call a interface to interact with database


In [ ]:
# @title
cursor.execute('''
CREATE TABLE IF NOT EXISTS persons (
  id INTEGER PRIMARY KEY,
  first_name TEXT,
  last_name TEXT,
  age INTEGER
)
''')
# this execute command stays in the connection pipeline
# and finally applies when we apply commit

connection.commit()


In [ ]:
# @title
cursor.execute('''
INSERT INTO persons VALUES
(1,'Paul', 'Smith', 24),
(2,'Mark', 'Jhonson', 30),
(3,'Anne', 'Smith', 34)
''')
cursor.execute('''
SELECT * FROM persons
WHERE last_name = 'Smith'
''')
# rows = cursor.fetchall()
# print(rows)

connection.commit()
connection.close()

In [ ]:
# @title
class Person():
  def __init__(self, id_num=-1, first="", last="", age=-1):
    self.id_num = id_num
    self.first = first
    self.last = last
    self.age = age
    self.connection = sqlite3.connect('/content/mydata.db')
    self.cursor = self.connection.cursor()

  def load_person(self, id_num):
    self.cursor.execute('''
    SELECT * FROM persons
    WHERE id = {}
    '''.format(id_num))

    results = self.cursor.fetchone()

    self.id_num = id_num
    self.first = results[1]
    self.last = results[2]
    self.age = results[3]

  def insert_person(self):
    self.cursor.execute('''
    INSERT INTO persons VALUES
    ("{}", "{}", "{}", {})
    '''.format(self.id_num, self.first, self.last, self.age))
    self.connection.commit()
    self.connection.close()

In [ ]:
# @title
# Load a person from the database
p1 = Person()
p1.load_person(1)
print(p1.first)
print(p1.last)
print(p1.age)
print(p1.id_num)

Paul
Smith
24
1


In [ ]:
# @title
# insert a person into the database
p1 = Person(7, "Alex", "Robins", 37)
p1.insert_person()

In [ ]:
# @title
connection = sqlite3.connect('/content/mydata.db')
cursor = connection.cursor()
cursor.execute("SELECT * FROM persons")
results = cursor.fetchall()
print(results)


[(1, 'Paul', 'Smith', 24), (2, 'Mark', 'Jhonson', 30), (3, 'Anne', 'Smith', 34), (7, 'Alex', 'Robins', 37)]


In [ ]:
connection.close()

In [ ]:
# @title Recursion

# with loop-> 5! = 5 * 4 * 3 * 2 * 1
n = 7
fact = 1
while n > 0:
  fact *= n
  n -= 1
print(fact)

5040


In [ ]:
# @title
# with recursion->
# 5! = 5 * 4! -> 5*4*3! -> 5*4*3*2! -> 5*4*3*2*1!
number = 1
def factorial(n):
  if n < 1:
    return 1
  else:
    number = n * factorial(n-1)
    #        7 * 6!
    return number
# n < 1 that recursed function returns 1,
# n = 1 that recursed function returns 1
# n = 2 that recursed function returns 2,
# n = 3 that recursed function returns 3
# n = 4 that recursed function returns 4,
# n = 5 that recursed function returns 5
# n = 6 that recursed function returns 6,
# n = 7 that function returns 7 -> and this one was the initial function call
# now they multiply
print(factorial(7))

5040


In [ ]:
# @title
# fibonacchi
def fibonacci_loop(n):
  a, b = 0, 1
  for x in range(n):
    a, b = b , a+b
  return a

fibo = 0
def fibonacci(n):
  if n <= 1:
    return n
  else:
    return (fibonacci(n-1) + fibonacci(n-2))

# initial call fibonacci(7)
# 7 -> f(6) + f(5) -> f(5)+f(4) + f(4)+f(3)
# -> f(4)+f(3) + f(3)+f(2) + f(3)+f(2) + f(2)+f(1)
# -> f(3)+f(2) + f(2)+f(1) + f(2)+f(1) + f(1)+f(0) + f(2)+f(1) + f(1)+f(0) + f(1)+f(0) + 1
# ->
# ->

In [ ]:
print(fibonacci(7))

13


In [ ]:
print(fibonacci_loop(7))

13


In [ ]:
# @title XML(Extensible Markup Language) processing
# alot of uses like building GUI's
#
# platform and application independent database system
# we can insert data into XML using python, then read it using JAVA vice-versa
# SAX module -> simple API for XML (limited manipulatibility) (use when we have limited memory/input)
# doesn't load the entire XML file to RAM
# DOM module -> Document Object Model

In [ ]:
import xml.sax

In [ ]:
# @title SAX
# handler-> handles XML file
# handler = xml.sax.ContentHandler()
class groupHandler(xml.sax.ContentHandler):

# we didn't define an __init__ method because this class will be
# initialized by xml.sax.ContentHandler class
# and we are changing a few methods within that class e.g. startElement, characters, endElement
  def startElement(self, name, attribute):
    #print(name)
    self.current = name

    if self.current == "person":
      print("---------PERSON-----------")
      print("ID: {}".format(attribute['id']))

  def characters(self, content):
    if self.current == "name":
      self.name = content
    elif self.current == "age":
      self.age = content
    elif self.current == "weight":
      self.weight = content
    elif self.current == "height":
      self.height = content

  def endElement(self, name):
    if self.current == "name":
      print("Name: {}".format(self.name))
    elif self.current == "age":
      print("Age: {}".format(self.age))
    elif self.current == "weight":
      print("Weight: {}".format(self.weight))
    elif self.current == "height":
      print("Height: {}".format(self.height))
    self.current =""

# parser -> translates the XML file to python

In [ ]:
# @title
handler = groupHandler()
parser = xml.sax.make_parser()
parser.setContentHandler(handler)
parser.parse("/content/data.xml")

In [ ]:
import xml.dom.minidom

In [ ]:
# @title DOM
# DOM views data as a tree structure
# so for our example data.xml -> root = group, branch = person, leaf = name, age, weight, height
domtree = xml.dom.minidom.parse("/content/data.xml")
group = domtree.documentElement

persons = group.getElementsByTagName("person")
for person in persons:
  print("-------------PERSON---------------")
  if person.hasAttribute('id'):
    print("ID: {}".format(person.getAttribute('id')))

  print("Name: {}".format(person.getElementsByTagName("name")[0].childNodes[0].data))
  print("Age: {}".format(person.getElementsByTagName("age")[0].childNodes[0].data))
  print("Weight: {}".format(person.getElementsByTagName("weight")[0].childNodes[0].data))
  print("Height: {}".format(person.getElementsByTagName("height")[0].childNodes[0].data))

In [ ]:
# @title #####change entries/data of the XML file
# change entries/data of the XML file
persons[2].getElementsByTagName('name')[0].childNodes[0].nodeValue = "New Name"
persons[0].setAttribute('id', '100')
persons[1].getElementsByTagName('age')[0].childNodes[0].nodeValue = "2000"

In [ ]:
domtree.writexml(open('/content/data.xml', 'w'))

In [ ]:
# @title #####create new entries
# create new entries
newperson = domtree.createElement('person')
newperson.setAttribute('id', '5')

name = domtree.createElement('name')
name.appendChild(domtree.createTextNode('Paul Green'))
age = domtree.createElement('age')
age.appendChild(domtree.createTextNode('22'))
weight = domtree.createElement('weight')
weight.appendChild(domtree.createTextNode('60'))
height = domtree.createElement('height')
height.appendChild(domtree.createTextNode('170'))

newperson.appendChild(name)
newperson.appendChild(age)
newperson.appendChild(weight)
newperson.appendChild(height)

group.appendChild(newperson)

In [ ]:
domtree.writexml(open('/content/data.xml', 'w'))

In [ ]:
# @title Logging
# logging helps to find problems, to avoid problems, to understand problems
# so when some programme doesn't work log messages will help to understand why it didn'dt woek
# and someone who knows what they are doing can look at the log and understand & fix the problem
# think of a computer os, there are a lot of log files being created behind the scenes
# different messages can be given different priorty levels

# 5 security levels
# DEBUG- mainly used by developers to play around and fix bugs
# INFO- info messages like 'you have 17 mails', '2 users online' etc
# WARNING- nothing bad happened yet, but will happen if current situation persists e.g. 'You are running low on memory'
# ERROR- System keeps running but process halts to give error e.g.'couldn't perform an action because xyz reason'
# CRITICAL- When an essential part of your system is in danger e.g. 'server is down'

# python can set the security level.
# So if security level=DEBUG, then python will show all log messages(DEBUG, INFO, WARNING, ERROR, CRITICAL)
# if security level=Warning, then log shown=(WARNING, ERROR, CRITICAL)



In [ ]:
import logging

In [ ]:
logging.basicConfig(level=logging.DEBUG)
logging.info("You have 20 mails in your inbox")
logging.critical("All system components have failed!")

CRITICAL:root:All system components have failed!


In [ ]:
# @title #####create your own logger
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger("MEME logger")
logger.info("Yo ho! I see you have summoned me, the MEME logger ")
logger.log(logging.ERROR, "An error occured!")
logger.critical("imma destroy your system now")

ERROR:MEME logger:An error occured!
CRITICAL:MEME logger:imma destroy your system now


In [ ]:
# @title #####Logger File Handler
# The previous log messages will continously keep changing on a system console acccording to situation
# but as a system admin, I will only want to look at the LOG FILES when something fails; not continously stare at log messages
logger.setLevel(logging.DEBUG) # logger will print down to DEBUG level messages

handler = logging.FileHandler("/content/Mylog.log")
handler.setLevel(logging.INFO) # but in LOG files only down to INFO level message will exist

formatter = logging.Formatter("%(levelname)s - %(asctime)s: %(message)s") # %(levelname)s -%(as)... these are keywords and not set by user
handler.setFormatter(formatter)

logger.addHandler(handler)

logger.debug("This is a debug message")
logger.info("This is info message")




DEBUG:MEME logger:This is a debug message
INFO:MEME logger:This is info message
